<h1 align=center><font size = 5>Boroughs and cities comparaison</font></h1>

<h1>Introduction</h1>
In this notebook we will use Foursquare API to explore boroughs of Paris, New-York, Toronto and Montreal. We will group those borough into cluster in order to find similarity and dissimilarity of those cities. We will use this work to find out which borough is the closet to Paris 11th arrondissement (borough)
   

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size = 3>
    
1. <a href="#item1">Import main libraries</a>
    
2. <a href="#item2">Data collection and formating</a>
        
3. <a href="#item3">Collecting foursquare data for each city</a>
    
4. <a href="#item4">Clustering boroughs</a>
   
    
</font>
</div>

## 1. Import main librairies

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')




Solving environment: done

# All requested packages already installed.

Libraries imported.


## 2. Data collection and formating

# 2.1Paris dataset
This dataset has been created by myself to list all Paris boroughs and neighborhoods with their postal code

In [2]:
paris_data_df = pd.read_excel('ParisData.xlsx')
# keep only postal code and arrondissement (borough)
paris_data_df = paris_data_df[['PostalCode','Borough']]
# adding city name : Paris
paris_data_df['City'] = 'Paris'
paris_data_df = paris_data_df.drop_duplicates().reset_index(drop=True)
paris_data_df.head()

,PostalCode,Borough,City
0,75001,Le Louvre,Paris
1,75002,La Bourse,Paris
2,75003,Le Temple,Paris
3,75004,L'Hôtel-de-Ville,Paris
4,75005,Le Panthéon,Paris


In [3]:
geo_paris_df = pd.DataFrame(columns=['PostalCode', 'latitude', 'longitude'])
geo_paris_df = geo_paris_df.astype({'PostalCode': int})
## getting latitude and longitude of each Paris borough
geolocator = Nominatim(user_agent="my_explorer")
for  pcode  in paris_data_df['PostalCode']: 
    address = '{}'.format(pcode)
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    geo_paris_df = geo_paris_df.append({'PostalCode' : pcode,
                                        'latitude' : latitude,
                                        'longitude' :longitude },ignore_index=True)
#Enhance paris_data with latitude and longitude
paris_data_df = paris_data_df.merge(geo_paris_df, left_on='PostalCode', right_on='PostalCode')
paris_data_df= paris_data_df.drop('PostalCode', axis=1)
paris_data_df.head()

,Borough,City,latitude,longitude
0,Le Louvre,Paris,48.857366,2.336815
1,La Bourse,Paris,48.867485,2.344107
2,Le Temple,Paris,48.861751,2.361134
3,L'Hôtel-de-Ville,Paris,48.855967,2.357213
4,Le Panthéon,Paris,48.852752,2.346343


In [89]:
paris_data_df

,Borough,City,latitude,longitude
0,Le Louvre,Paris,48.857366,2.336815
1,La Bourse,Paris,48.867485,2.344107
2,Le Temple,Paris,48.861751,2.361134
3,L'Hôtel-de-Ville,Paris,48.855967,2.357213
4,Le Panthéon,Paris,48.852752,2.346343
5,Le Luxembourg,Paris,32.965034,-96.900153
6,Le Palais-Bourbon,Paris,48.855966,2.313815
7,L'Élysée,Paris,48.868853,2.320879
8,L'Opéra,Paris,48.877350,2.336920
9,L'Enclos-Saint-Laurent,Paris,48.879201,2.354391


# 2.2 Newyork dataset

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
newyork_data_list = newyork_data['features']

column_names = ['Borough','City'] 
newyork_data_df = pd.DataFrame(columns=column_names)

for data in newyork_data_list:
    borough  = data['properties']['borough'] 
    newyork_data_df = newyork_data_df.append({'Borough': borough, 'City': 'New York'}, ignore_index=True)
newyork_data_df = newyork_data_df.drop_duplicates().reset_index(drop=True)
newyork_data_df.head()

Data downloaded!


,Borough,City
0,Bronx,New York
1,Manhattan,New York
2,Brooklyn,New York
3,Queens,New York
4,Staten Island,New York


In [5]:
geo_newyork_df =pd.DataFrame(columns=['Borough', 'latitude', 'longitude'])
geolocator = Nominatim(user_agent="my_explorer")
for  borough  in newyork_data_df['Borough']: 
    address = '{},{}'.format(borough,'New York')
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    geo_newyork_df = geo_newyork_df.append({'Borough' : borough,
                                        'latitude' : latitude,
                                        'longitude' :longitude },ignore_index=True)

##Enhance newyork_data with latitude and longitude
newyork_data_df = newyork_data_df.merge(geo_newyork_df, left_on='Borough', right_on='Borough')
newyork_data_df.head()


,Borough,City,latitude,longitude
0,Bronx,New York,40.846651,-73.878594
1,Manhattan,New York,40.789624,-73.959894
2,Brooklyn,New York,40.650104,-73.949582
3,Queens,New York,40.749824,-73.797634
4,Staten Island,New York,40.583456,-74.149605


# 2.3 Toronto Dataset

#### Install lxml for data scraping

In [8]:
conda install -c conda-forge lxml

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libxslt-1.1.33             |       h7d1a2b0_0         426 KB
    lxml-3.8.0                 |           py36_0         3.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following NEW packages will be INSTALLED:

  libxslt            pkgs/main/linux-64::libxslt-1.1.33-h7d1a2b0_0
  lxml               conda-forge/linux-64::lxml-3.8.0-py36_0



lxml-3.8.0           | 3.8 MB    | ##################################### | 100% 
libxslt-1.1.33       | 426 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you ma

In [6]:
# Scraping data from wikipedia to create the dataFrame
toronto_data_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
# Adding city name : Toronto
toronto_data_df['City'] = 'Toronto'
# Remove not assigned code
toronto_data_df = toronto_data_df[toronto_data_df['Borough']!='Not assigned'].reset_index(drop=True)
# Remove not necessary columns and remove duplicates
toronto_data_df = toronto_data_df.drop(['Neighborhood','Postal Code'],axis=1)
toronto_data_df = toronto_data_df.drop_duplicates()
toronto_data_df.head()

,Borough,City
0,North York,Toronto
2,Downtown Toronto,Toronto
5,Etobicoke,Toronto
6,Scarborough,Toronto
8,East York,Toronto


In [7]:
geo_toronto_df =pd.DataFrame(columns=['Borough', 'latitude', 'longitude'])
geolocator = Nominatim(user_agent="my_explorer")
for  borough, city  in zip(toronto_data_df['Borough'],toronto_data_df['City']): 
    address = '{},{}'.format(borough,city)
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    geo_toronto_df = geo_toronto_df.append({'Borough' : borough,
                                        'latitude' : latitude,
                                        'longitude' :longitude },ignore_index=True)

##Enhance toront_data with latitude and longitude
toronto_data_df = toronto_data_df.merge(geo_toronto_df, left_on='Borough', right_on='Borough')
toronto_data_df.head()


,Borough,City,latitude,longitude
0,North York,Toronto,43.754326,-79.449117
1,Downtown Toronto,Toronto,43.654174,-79.380812
2,Etobicoke,Toronto,43.643556,-79.565633
3,Scarborough,Toronto,43.773077,-79.257774
4,East York,Toronto,43.699971,-79.332520


## 2.4 Montreal dataset

In [8]:
!wget -q -O 'montreal.json' http://donnees.ville.montreal.qc.ca/dataset/00bd85eb-23aa-4669-8f1b-ba9a000e3dd8/resource/e9b0f927-8f75-458c-8fda-b5da65cc8b73/download/limadmin.json
print('Data downloaded!')


Data downloaded!


In [9]:
with open('montreal.json') as json_data:
    montreal_data = json.load(json_data)
montreal_data_list = montreal_data['features']

In [10]:
montreal_data_list[0]['properties']

{'NOM': 'Outremont',
 'TYPE': 'Arrondissement',
 'CODEID': '11',
 'ABREV': 'OM',
 'NUM': 5,
 'CODEMAMROT': 'REM05',
 'AIRE': 3813355.72326504,
 'MUNID': 66023,
 'PERIM': 10836.6706340882}

In [11]:
column_names = ['Borough','City'] 
montreal_data_df = pd.DataFrame(columns=column_names)

for data in montreal_data_list:
    borough  = data['properties']['NOM'] 
    city = 'Montreal'

    
    montreal_data_df = montreal_data_df.append({'Borough': borough, 'City': city}, ignore_index=True)
montreal_data_df = montreal_data_df.drop_duplicates().reset_index(drop=True)
montreal_data_df.head()

,Borough,City
0,Outremont,Montreal
1,LaSalle,Montreal
2,Mont-Royal,Montreal
3,Ville-Marie,Montreal
4,Le Plateau-Mont-Royal,Montreal


In [12]:
geo_montreal_df = pd.DataFrame(columns=['Borough','latitude','longitude'])

geolocator = Nominatim(user_agent="my_explorer")
for borough, city  in zip(montreal_data_df['Borough'],montreal_data_df['City']): 
    address = '{},{}'.format(borough,city)
    location = geolocator.geocode(address)
    if (location is None):
        latitude = 'NA'
        longitude = 'NA'
    else:
        latitude = location.latitude
        longitude = location.longitude
    geo_montreal_df = geo_montreal_df.append({'Borough' : borough,
                                           'latitude' : latitude,
                                           'longitude' :longitude },ignore_index=True)

montreal_data_df =  montreal_data_df.merge(geo_montreal_df, left_on ='Borough', right_on ='Borough')
montreal_data_df.head()


,Borough,City,latitude,longitude
0,Outremont,Montreal,45.518617,-73.606886
1,LaSalle,Montreal,45.432514,-73.629267
2,Mont-Royal,Montreal,45.508616,-73.590311
3,Ville-Marie,Montreal,45.513444,-73.546535
4,Le Plateau-Mont-Royal,Montreal,45.521836,-73.582173


# Cities dataframe

In [14]:
cities_df = pd.DataFrame(columns=['City','Country','city_lat','city_lon'])
cities_df = cities_df.append({'City' : 'Paris', 'Country': 'France'},ignore_index=True)
cities_df = cities_df.append({'City' : 'New York', 'Country': 'United States'},ignore_index=True )
cities_df = cities_df.append( {'City' : 'Toronto', 'Country': 'Canada'},ignore_index=True )
cities_df = cities_df.append( {'City' : 'Montreal', 'Country': 'Canada'},ignore_index=True )
cities_df

,City,Country,city_lat,city_lon
0,Paris,France,NaN,NaN
1,New York,United States,NaN,NaN
2,Toronto,Canada,NaN,NaN
3,Montreal,Canada,NaN,NaN


In [15]:
# cities location
geolocator = Nominatim(user_agent="my_explorer")
for index in range (len(cities_df)):
    address = '{}, {}'.format(cities_df.loc[index,'City'],cities_df.loc[index,'Country'])
    location = geolocator.geocode(address)
    cities_df.loc[index, 'city_lat']= location.latitude
    cities_df.loc[index,'city_lon'] = location.longitude
cities_df

,City,Country,city_lat,city_lon
0,Paris,France,48.8567,2.35146
1,New York,United States,40.7127,-74.006
2,Toronto,Canada,43.6535,-79.3839
3,Montreal,Canada,45.4972,-73.6104


### Gathering the cities dataframe into one boroughs_df

In [16]:
boroughs_df = pd.DataFrame(columns=['Borough','City','latitude','longitude'])

In [17]:
boroughs_df = boroughs_df.append(paris_data_df, ignore_index = True)
boroughs_df = boroughs_df.append(newyork_data_df, ignore_index = True)
boroughs_df = boroughs_df.append(toronto_data_df, ignore_index = True)
boroughs_df = boroughs_df.append(montreal_data_df, ignore_index = True)

In [18]:
boroughs_df.head()

,Borough,City,latitude,longitude
0,Le Louvre,Paris,48.857366,2.336815
1,La Bourse,Paris,48.867485,2.344107
2,Le Temple,Paris,48.861751,2.361134
3,L'Hôtel-de-Ville,Paris,48.855967,2.357213
4,Le Panthéon,Paris,48.852752,2.346343


# 3. Collecting foursquare data for each city

In [115]:
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = ' your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT =100
radius =500


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
borough_venues = getNearbyVenues(names=boroughs_df['Borough'],
                                   latitudes=boroughs_df['latitude'],
                                   longitudes=boroughs_df['longitude'], 
                                   radius = radius
                                  )


In [22]:
borough_venues.shape

(1948, 7)

In [23]:
borough_venues.head() 

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Le Louvre,48.857366,2.336815,Buly 1803,48.857146,2.335039,Cosmetics Shop
1,Le Louvre,48.857366,2.336815,Pont des Arts,48.858565,2.337635,Bridge
2,Le Louvre,48.857366,2.336815,L'Hotel,48.856377,2.335193,Hotel
3,Le Louvre,48.857366,2.336815,Guy Savoy,48.857085,2.338807,French Restaurant
4,Le Louvre,48.857366,2.336815,Monnaie de Paris,48.856833,2.339058,Museum


In [25]:
# one hot encoding
borough_onehot = pd.get_dummies(borough_venues[['Venue Category']], prefix="", prefix_sep="")

borough_onehot.head()

,Accessories Store,African Restaurant,Airport,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basque Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Colombian Restaurant,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,EV Charging Station,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Market,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Portuguese Restaurant,Print Shop,Pub,Racecourse,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,River,Road,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Train,Train Station,Transportation Service,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [26]:
# add borough column back to dataframe
borough_onehot['Borough'] = borough_venues['Borough'] 

In [27]:
fixed_columns = [borough_onehot.columns[-1]] + list(borough_onehot.columns[:-1])
borough_onehot = borough_onehot[fixed_columns]
borough_onehot.head()

,Borough,Accessories Store,African Restaurant,Airport,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basque Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Colombian Restaurant,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,EV Charging Station,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Market,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Portuguese Restaurant,Print Shop,Pub,Racecourse,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,River,Road,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Train,Train Station,Transportation Service,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Le Louvre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [28]:
borough_grouped = borough_onehot.groupby('Borough').mean().reset_index()
borough_grouped.head()

,Borough,Accessories Store,African Restaurant,Airport,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basque Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Colombian Restaurant,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,EV Charging Station,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Market,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Portuguese Restaurant,Print Shop,Pub,Racecourse,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,River,Road,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Train,Train Station,Transportation Service,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Ahuntsic-Cartierville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [29]:
borough_grouped.shape

(67, 277)

In [30]:
num_top_venues = 2

for borough in borough_grouped['Borough']:
    print("----"+borough+"----")
    temp = borough_grouped[borough_grouped['Borough'] == borough].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ahuntsic-Cartierville----
                venue  freq
0  Italian Restaurant  0.12
1        Hockey Arena  0.12


----Anjou----
                   venue  freq
0  Portuguese Restaurant  0.33
1           Dance Studio  0.33


----Baie-d'Urfé----
           venue  freq
0  Grocery Store  0.25
1   Liquor Store  0.25


----Beaconsfield----
          venue  freq
0   Pizza Place   0.4
1  Soccer Field   0.2


----Bronx----
         venue  freq
0  Zoo Exhibit  0.28
1          Zoo  0.12


----Brooklyn----
                  venue  freq
0  Caribbean Restaurant  0.24
1             Juice Bar  0.07


----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Clothing Store  0.08


----Côte-Saint-Luc----
                           venue  freq
0                            Gym  0.22
1  Paper / Office Supplies Store  0.11


----Côte-des-Neiges-Notre-Dame-de-Grâce----
         venue  freq
0  Coffee Shop  0.09
1     Pharmacy  0.09


----Dollard-des-Ormeaux----
                venue  freq
0 

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Borough_venues_sorted = pd.DataFrame(columns=columns)
Borough_venues_sorted['Borough'] = borough_grouped['Borough']

for ind in np.arange(borough_grouped.shape[0]):
    Borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(borough_grouped.iloc[ind, :], num_top_venues)

Borough_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ahuntsic-Cartierville,Sandwich Place,Café,Hockey Arena,Chinese Restaurant,Italian Restaurant,Park,Middle Eastern Restaurant,Train Station,Electronics Store,Empanada Restaurant
1,Anjou,Portuguese Restaurant,Supermarket,Dance Studio,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market
2,Baie-d'Urfé,Sandwich Place,Train Station,Grocery Store,Liquor Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Drugstore
3,Beaconsfield,Pizza Place,Hockey Arena,Soccer Field,Business Service,Zoo Exhibit,Falafel Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
4,Bronx,Zoo Exhibit,Park,Zoo,Souvenir Shop,Fast Food Restaurant,Fried Chicken Joint,Scenic Lookout,Garden,Gas Station,General Travel


# 4 Clustering boroughs

In [98]:
# set number of clusters
kclusters = 5

borough_grouped_clustering = borough_grouped.drop('Borough', 1)

borough_grouped_clustering.head()

,Accessories Store,African Restaurant,Airport,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basque Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Colombian Restaurant,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,EV Charging Station,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Market,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Portuguese Restaurant,Print Shop,Pub,Racecourse,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,River,Road,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Train,Train Station,Transportation Service,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [99]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(borough_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [100]:
Borough_venues_sorted = Borough_venues_sorted.drop('Cluster', axis=1)

In [101]:
# add clustering labels
Borough_venues_sorted.insert(0, 'Cluster', kmeans.labels_)
Borough_venues_sorted.head()


,Cluster,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Ahuntsic-Cartierville,Sandwich Place,Café,Hockey Arena,Chinese Restaurant,Italian Restaurant,Park,Middle Eastern Restaurant,Train Station,Electronics Store,Empanada Restaurant
1,3,Anjou,Portuguese Restaurant,Supermarket,Dance Studio,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market
2,0,Baie-d'Urfé,Sandwich Place,Train Station,Grocery Store,Liquor Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Drugstore
3,0,Beaconsfield,Pizza Place,Hockey Arena,Soccer Field,Business Service,Zoo Exhibit,Falafel Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
4,0,Bronx,Zoo Exhibit,Park,Zoo,Souvenir Shop,Fast Food Restaurant,Fried Chicken Joint,Scenic Lookout,Garden,Gas Station,General Travel


In [102]:
borough_merged = boroughs_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
borough_merged = borough_merged.join(Borough_venues_sorted.set_index('Borough'), on='Borough')

In [103]:
##Cluster 1
borough_merged.loc[borough_merged['Cluster'] == 0, borough_merged.columns[[0,1] + list(range(5, borough_merged.shape[1]))]]

,Borough,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Le Luxembourg,Paris,Electronics Store,Park,Water Park,Baseball Field,Falafel Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
15,Passy,Paris,Circus,Plaza,Racecourse,Nature Preserve,Snack Place,Lake,Zoo Exhibit,Empanada Restaurant,Ethiopian Restaurant,Event Space
19,Ménilmontant,Paris,Mexican Restaurant,Hotel,Burger Joint,Sandwich Place,Fried Chicken Joint,Big Box Store,Gas Station,Farmers Market,Event Space,Exhibit
20,Bronx,New York,Zoo Exhibit,Park,Zoo,Souvenir Shop,Fast Food Restaurant,Fried Chicken Joint,Scenic Lookout,Garden,Gas Station,General Travel
21,Manhattan,New York,Park,Baseball Field,Playground,Breakfast Spot,Bus Station,Metro Station,Bus Stop,Field,Bike Rental / Bike Share,Tennis Court
22,Brooklyn,New York,Caribbean Restaurant,Juice Bar,Convenience Store,Bakery,Pharmacy,Donut Shop,Chinese Restaurant,Sandwich Place,School,Café
23,Queens,New York,Korean Restaurant,Sports Club,Boat or Ferry,Bus Station,Bike Trail,Zoo Exhibit,Exhibit,Factory,Falafel Restaurant,Farmers Market
24,Staten Island,New York,Trail,Sandwich Place,Business Service,EV Charging Station,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
25,North York,Toronto,Coffee Shop,Mediterranean Restaurant,Pizza Place,Middle Eastern Restaurant,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
26,Downtown Toronto,Toronto,Coffee Shop,Clothing Store,Hotel,Diner,Middle Eastern Restaurant,Restaurant,Ramen Restaurant,Theater,Bar,Plaza


In [104]:
##Cluster 2
borough_merged.loc[borough_merged['Cluster'] == 1, borough_merged.columns[[0,1] + list(range(5, borough_merged.shape[1]))]]

,Borough,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Sainte-Anne-de-Bellevue,Montreal,Gym,Train Station,Zoo Exhibit,EV Charging Station,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant


In [105]:
##Cluster 3
borough_merged.loc[borough_merged['Cluster'] == 2, borough_merged.columns[[0,1] + list(range(5, borough_merged.shape[1]))]]

,Borough,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,L'Île-Bizard-Sainte-Geneviève,Montreal,Golf Course,Zoo Exhibit,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Fast Food Restaurant,Electronics Store


In [107]:
##Cluster 4
borough_merged.loc[borough_merged['Cluster'] == 3, borough_merged.columns[[0,1] + list(range(5, borough_merged.shape[1]))]]

,Borough,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Le Louvre,Paris,French Restaurant,Hotel,Café,Plaza,Bookstore,Japanese Restaurant,Ice Cream Shop,Restaurant,Boutique,Tea Room
1,La Bourse,Paris,French Restaurant,Wine Bar,Cocktail Bar,Italian Restaurant,Bakery,Hotel,Salad Place,Japanese Restaurant,Tea Room,Indie Movie Theater
2,Le Temple,Paris,French Restaurant,Italian Restaurant,Sandwich Place,Wine Bar,Coffee Shop,Gourmet Shop,Japanese Restaurant,Clothing Store,Burger Joint,Garden
3,L'Hôtel-de-Ville,Paris,French Restaurant,Clothing Store,Plaza,Ice Cream Shop,Hotel,Italian Restaurant,Wine Bar,Pedestrian Plaza,Gourmet Shop,Café
4,Le Panthéon,Paris,French Restaurant,Hotel,Plaza,Indie Movie Theater,Coffee Shop,Bookstore,Bakery,Bar,Flower Shop,Burger Joint
6,Le Palais-Bourbon,Paris,French Restaurant,Hotel,Café,History Museum,Plaza,Italian Restaurant,Park,Historic Site,Art Museum,Garden
7,L'Élysée,Paris,Hotel,French Restaurant,Boutique,Women's Store,Garden,Gourmet Shop,Dessert Shop,Clothing Store,Bookstore,Tea Room
8,L'Opéra,Paris,French Restaurant,Hotel,Cocktail Bar,Bistro,Bakery,Lounge,Japanese Restaurant,Bar,Theater,Pizza Place
9,L'Enclos-Saint-Laurent,Paris,French Restaurant,Hotel,Restaurant,Indian Restaurant,Coffee Shop,Japanese Restaurant,Café,Seafood Restaurant,Gastropub,Bistro
10,Popincourt,Paris,French Restaurant,Bar,Hotel,Cocktail Bar,Wine Bar,Vietnamese Restaurant,Bistro,Record Shop,Beer Store,Noodle House


In [106]:
##Cluster 5
borough_merged.loc[borough_merged['Cluster'] == 4, borough_merged.columns[[0,1] + list(range(5, borough_merged.shape[1]))]]

,Borough,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
55,L'Île-Dorval,Montreal,Boat or Ferry,Zoo Exhibit,Field,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [79]:
borough_merged.head()

,Borough,City,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Le Louvre,Paris,48.857366,2.336815,0.0,French Restaurant,Hotel,Café,Plaza,Bookstore,Japanese Restaurant,Ice Cream Shop,Restaurant,Boutique,Tea Room
1,La Bourse,Paris,48.867485,2.344107,0.0,French Restaurant,Wine Bar,Cocktail Bar,Italian Restaurant,Bakery,Hotel,Salad Place,Japanese Restaurant,Tea Room,Indie Movie Theater
2,Le Temple,Paris,48.861751,2.361134,0.0,French Restaurant,Italian Restaurant,Sandwich Place,Wine Bar,Coffee Shop,Gourmet Shop,Japanese Restaurant,Clothing Store,Burger Joint,Garden
3,L'Hôtel-de-Ville,Paris,48.855967,2.357213,0.0,French Restaurant,Clothing Store,Plaza,Ice Cream Shop,Hotel,Italian Restaurant,Wine Bar,Pedestrian Plaza,Gourmet Shop,Café
4,Le Panthéon,Paris,48.852752,2.346343,0.0,French Restaurant,Hotel,Plaza,Indie Movie Theater,Coffee Shop,Bookstore,Bakery,Bar,Flower Shop,Burger Joint


In [108]:
borough_merged1 =  borough_merged.dropna()

In [109]:
borough_merged1 = borough_merged1.astype({'Cluster' : int})

In [111]:
borough_merged1 

,Borough,City,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Le Louvre,Paris,48.857366,2.336815,3,French Restaurant,Hotel,Café,Plaza,Bookstore,Japanese Restaurant,Ice Cream Shop,Restaurant,Boutique,Tea Room
1,La Bourse,Paris,48.867485,2.344107,3,French Restaurant,Wine Bar,Cocktail Bar,Italian Restaurant,Bakery,Hotel,Salad Place,Japanese Restaurant,Tea Room,Indie Movie Theater
2,Le Temple,Paris,48.861751,2.361134,3,French Restaurant,Italian Restaurant,Sandwich Place,Wine Bar,Coffee Shop,Gourmet Shop,Japanese Restaurant,Clothing Store,Burger Joint,Garden
3,L'Hôtel-de-Ville,Paris,48.855967,2.357213,3,French Restaurant,Clothing Store,Plaza,Ice Cream Shop,Hotel,Italian Restaurant,Wine Bar,Pedestrian Plaza,Gourmet Shop,Café
4,Le Panthéon,Paris,48.852752,2.346343,3,French Restaurant,Hotel,Plaza,Indie Movie Theater,Coffee Shop,Bookstore,Bakery,Bar,Flower Shop,Burger Joint
5,Le Luxembourg,Paris,32.965034,-96.900153,0,Electronics Store,Park,Water Park,Baseball Field,Falafel Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
6,Le Palais-Bourbon,Paris,48.855966,2.313815,3,French Restaurant,Hotel,Café,History Museum,Plaza,Italian Restaurant,Park,Historic Site,Art Museum,Garden
7,L'Élysée,Paris,48.868853,2.320879,3,Hotel,French Restaurant,Boutique,Women's Store,Garden,Gourmet Shop,Dessert Shop,Clothing Store,Bookstore,Tea Room
8,L'Opéra,Paris,48.877350,2.336920,3,French Restaurant,Hotel,Cocktail Bar,Bistro,Bakery,Lounge,Japanese Restaurant,Bar,Theater,Pizza Place
9,L'Enclos-Saint-Laurent,Paris,48.879201,2.354391,3,French Restaurant,Hotel,Restaurant,Indian Restaurant,Coffee Shop,Japanese Restaurant,Café,Seafood Restaurant,Gastropub,Bistro


In [113]:
# create map
map_clusters = folium.Map(location=[45, -45], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(borough_merged1['latitude'], borough_merged1['longitude'], borough_merged1['Borough'], borough_merged1['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters